# First steps
Please make sure, that you have set up your environment. Follow the instructions inside the README.md,
Afterwards start the database by running `docker compose up` in one terminal. Install the `3dcitydb Importer/Exporter` https://github.com/3dcitydb/importer-exporter and connect to the database. After that, load the EnergyAde into the database. Lastly import a gml-file into the database using the Importer/Exporter.


To import EnergyDemand data into the database in a first step the data has to be prepared. For that, 2 data columns are read from a csv-file  together with the time-column. 

In [7]:
import csv
import os
from random import choice
from asgiref.sync import sync_to_async

import django
from django.core.management import call_command

# from citydb.models import Building


# open the example CSV-file and create a python dictionary:

dictOfEnergyDemandData = {}
columnDate = []
energyADEData = []
energyADEData2 = []
# Works only on first run, otherwise the env is changed and path is not relative any longer
with open('B1005.csv', 'r') as file:
    # Create a CSV reader object
    reader = csv.reader(file)
    headings = reader.__next__()
    for row in reader:
        columnDate.append(row[headings.index("DATE")])
        energyADEData.append(row[headings.index("GRID_kWh")])
        energyADEData2.append(row[headings.index("E_sys_kWh")])


# set the django-env variable and import the settings
os.chdir("../../")
os.environ['DJANGO_SETTINGS_MODULE'] = 'vdistrict.vdistrict.settings'
django.setup()

dictOfEnergyDemandData["DATE"] = columnDate


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\felix\\Programmieren\\django-citydb\\B1005.csv'

The data is put into a dictionary, whereby the time-information is a list of strings, where the string has the form "Year-Month-Day Hour:Minutes:Seconds". Each data-column is saved as a list of floats. To access a data-column a key as to be specified. The key is built from the EnergyDemand-end_use, as which it should be imported and the unit of the data separated by a underscore.

Please note: Since the cells inside the python-notebook are executed in a async-manner but the custom management-commands can be only executed synchronisly the conversion `sync_to_async` has to be done. That is not needed when executing the command inside a .py-file.

Please note further, that the building, for which `EnergyDemand`-data is added, needs to be loaded beforehand.

In [2]:

# these choices are possible for the energydemand-enduse
energyDemandEndUseChoices = [ 
    "domesticHotWater",
    "electricalAppliances",
    "lighting",
    # "otherOrCombinationElectrical",
    # "otherOrCombinationHeating",
    # "otherOrCombinationCooling",
    "spaceCooling",
    # "spaceHeating",
    "ventilation",
    # "ventilationHeating",
    # "ventilationCooling",
    "process",
]
energyDemandEndUse = choice(energyDemandEndUseChoices)
energyDemandEndUseChoices.pop(energyDemandEndUseChoices.index(energyDemandEndUse))
energyDemandEndUse2 = choice(energyDemandEndUseChoices)

dictOfEnergyDemandData[f"{energyDemandEndUse}_kWh"] = energyADEData

dictOfEnergyDemandData[f"{energyDemandEndUse2}_kWh"] = energyADEData2
# to set the building for which the EnergyDemand should be imported, specify the building-gmlid


optionalMetaInformation = {
    "acquisition_method": "estimation",
    "source": "VDistrict",
    "interpolation_type": "averageInSucceedingInterval",
    "quality_description": "Your quality description",

}

await sync_to_async(call_command)('import_timeseries', dictOfEnergyDemandData, "BLDG_0003000f0032da1b", **optionalMetaInformation)



The time differences in the time series are not equal.
The time differences in the time series are not equal.


c:\Users\felix\.conda\envs\django_citydb\Lib\site-packages\django\db\models\fields\__init__.py:1659: RuntimeWarning: DateTimeField RegularTimeSeries.timeperiodprop_beginposition received a naive datetime (2020-01-01 00:00:00) while time zone support is active.
  warnings.warn(
c:\Users\felix\.conda\envs\django_citydb\Lib\site-packages\django\db\models\fields\__init__.py:1659: RuntimeWarning: DateTimeField RegularTimeSeries.timeperiodproper_endposition received a naive datetime (2020-12-31 23:00:00) while time zone support is active.
  warnings.warn(


In [3]:
dictOfEnergyDemandData[f"domesticHotWater_kWh"] = energyADEData
await sync_to_async(call_command)('import_timeseries', dictOfEnergyDemandData, "BLDG_0003000f0032da1b")
# call_command('import_timeseries', dictOfEnergyDemandData, "BLDG_0003000b0054e8af")


The time differences in the time series are not equal.
The time differences in the time series are not equal.
